In [1]:
# Courtney King 
# ML Semester Project 
# May 9 2022

In [2]:
import pandas as pd
import numpy as np
import math
import pickle
import numpy as np
import random

In [3]:
# cleaned and processed data sets 
daily_numerical_features = pd.read_csv('csv/daily_numerical_features.csv')
monthly_numerical_features = pd.read_csv('csv/monthly_numerical_features.csv')
multinomial_numerical_features = pd.read_csv('csv/multinomial_features.csv')
labels = pd.read_csv('csv/labels.csv')

# had to use pickle because the df / csv did not properly preserve types 
with open('multinomial_dict.pkl', 'rb') as f:
    multinomial_dict = pickle.load(f)

In [4]:
# melt years, April, May 
labels['month'].value_counts()

5    68
4    36
Name: month, dtype: int64

## 1) Bayes with Multinomial Distribution

### Table 1a.  The imported dataset of weather types over time

In [5]:
pd.DataFrame.from_dict(multinomial_dict)

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1,"[SN, SN, FG, FG, FG, FG, FG, FG, SN, SN, SN]","[BR, BR, SN, SN, BR]","[SN, SN, SN, BR, FG, BR, HZ, SN, BR, HZ, HZ, S...","[SN, SN, BR, HZ, BR, SN, BR, SN, BR, SN, BR, S...",[],[],[],[],"[HZ, SN, BR, SN, BR, SN, BR, HZ, BR, HZ, BR, H...","[SN, UP, SN, BR, UP, SN, BR, UP, SN, FZFG, BR,...",...,"[RA, BR, RA, RA, RA, RA, RA, SN, BR, RA, SN, H...","[RA, SN, RA, SN, SN, RA, RA, RA, SN, HZ, FZFG,...","[SN, BR, SN, SN, RA, RA, BR, RA, UP, RA, RA, S...","[BR, HZ, RA, RA, RA, SN, FZRA, SN, BR, HZ, RA,...","[SN, BR, SN, BR, SN, BR, UP, BR, SN, SN, BR, S...","[BR, BR, UP, FZRA, SN, BR, RA, FZRA, SN, FZFG,...","[RA, SN, FZFG, BR, UP, FG, BR, UP, HZ, HZ, BR,...","[SN, BR, UP, HZ, SN, HZ, HZ, SN, HZ, BR, HZ, B...","[HZ, SN, BR, UP, UP, BR, HZ, HZ, HZ, SN, SN, HZ]","[SN, UP, HZ]"
2,"[FG, FG, FG, FG, SN, FG, SN, SN, SN, SN, SN, SN]","[BR, SN, BR, BR, SN]","[SN, BR, HZ, RA, SN, BR, SN, BR, SN]","[SN, SN, HZ, SN, BR, HZ, SN, BR, HZ, SN, BR, H...",[],[],[],[],"[SN, BR, SN, BR, HZ, HZ, SN, BR, SN, SN, HZ, S...","[BR, HZ, BR, HZ, BR, UP, HZ, BR, HZ, BR, HZ, U...",...,"[RA, RA, RA, SN, BR, RA, BR, RA, SN, HZ, RA, S...","[UP, SN, BR, SN, BR, HZ, FZRA, SN, FZFG, BR, U...","[SN, BR, RA, HZ, RA, SN, RA, RA, RA, RA, HZ, R...","[RA, UP, RA, UP, SN, HZ, FZFG, BR, HZ, RA, FG,...","[FZFG, BR, UP, HZ, FG, FZFG, BR, HZ, FG, HZ, U...","[RA, RA, SN, BR, RA, BR, RA, SN, BR, UP, HZ, S...","[SN, BR, SN, BR, SN, BR, UP, RA, SN, HZ, BR, S...","[HZ, SN, HZ, SN, SN, FZFG, BR, HZ, FG, SN, BR,...","[SN, SN, HZ, SN, SN, SN, FG, BR, FZFG, BR, HZ,...",[]
3,"[SN, SN, SN, SN, SN, RA]","[SN, SN, SN]","[SN, SN, BR, SN, BR, SN, BR, SN, SN, SN]","[SN, BR, SN, BR, SN, BR, HZ, HZ, SN, SN, SN, H...",[],[],[],[],"[SN, BR, HZ, SN, BR, SN, FG, BR, HZ, SN, FG, B...","[SN, BR, UP, SN, UP, SN, UP, UP, UP, RA, UP, U...",...,"[SN, BR, RA, HZ, RA, BR, RA, SN, FZFG, BR, FG,...","[UP, SN, BR, HZ, SN, BR, HZ, SN, UP, UP, RA, U...","[SN, BR, RA, SN, BR, SN, BR, UP, SN, BR, UP, H...","[RA, SN, SN, SN, BR, HZ, HZ, UP, SN, HZ, RA, R...","[RA, RA, SN, SN, BR, UP, HZ, SN, BR, HZ, SN, B...","[HZ, SN, BR, UP, SN, FZFG, BR, FG, SN, BR, UP,...","[SN, BR, SN, BR, SN, BR, HZ, SN, BR, RA, SN, B...","[SN, SN, HZ, SN, HZ, HZ, HZ, SN, HZ, SN, HZ, S...","[SN, SN, SN, SN, HZ, SN, BR, HZ, SN, BR, SN, H...",[]
4,"[SN, SN, RA]","[SN, SN, SN, RA]","[SN, BR, RA, RA, SN, BR, SN, BR, SN]","[SN, BR, SN, BR, SN, FG, SN, RA, SN, BR]",[],[],[],[],"[SN, SN, SN, BR, RA, RA, RA]","[SN, UP, SN, BR, UP, SN, BR, UP, SN, BR, UP, U...",...,"[SN, BR, RA, SN, SN, BR, HZ, SN, BR, HZ, RA, S...","[RA, RA, RA, RA, SN, BR, HZ, RA, SN, BR, RA, R...","[SN, HZ, RA, SN, RA, SN, BR, HZ, UP, SN, SN, B...","[SN, BR, SN, BR, UP, RA, RA, RA, RA, SN, BR, H...","[RA, RA, HZ, RA, RA, SN, BR, RA, SN, BR, UP, H...","[SN, BR, SN, RA, RA, RA, RA, RA, RA, RA, RA, R...","[RA, RA, RA, RA, RA, RA, BR, HZ, RA, RA, SN, B...","[SN, BR, HZ, RA, SN, FG, BR, UP, RA, SN, UP, H...","[SN, UP, HZ, SN, BR, SN, FZFG, BR, FG, RA, SN,...",[]
5,"[RA, FG, RA, RA]","[RA, RA, RA, RA, RA, RA]","[RA, SN, BR, SN, BR, RA, RA, RA, RA, BR, RA, R...","[SN, BR, RA, RA, RA, FG, BR, RA, RA, RA, FG, B...",[],[],[],[],"[RA, BR, RA, BR, RA, SN, BR, RA, BR, RA, BR, R...","[RA, HZ, RA, BR, HZ, BR, BR]",...,"[SN, BR, HZ, RA, SN, RA, BR, SN, FZFG, BR, UP,...","[RA, RA, RA, RA, HZ, BR, RA, RA, RA, RA, RA, R...","[RA, BR, BR, RA, RA, RA]","[RA, RA, BR, RA, RA, RA, RA, RA, BR, RA, RA, R...","[RA, RA, RA, RA, RA, RA, RA, RA, RA, BR, RA, R...","[RA, SN, BR, UP, SN, BR, RA, RA, RA, BR, RA, S...","[RA, SN, BR, RA, RA, RA, RA, RA, RA, RA, RA, F...","[RA, RA, RA, RA]","[RA, RA, RA, RA, RA, RA, RA, RA, RA]",[]
6,"[RA, SN]",[RA],"[RA, RA, RA, RA, RA, RA, RA, FG, BR, RA, FG, B...","[RA, BR, RA, RA, RA, RA, RA, BR, RA, BR, RA, F...",[],[],[],[],"[RA, RA, BR, FG, BR, RA, BR, RA, RA, RA, FG, B...","[HZ, BR, FG, BR, HZ, BR, HZ, BR, BR, BR, HZ, H...",...,"[RA, RA, RA, RA, RA, RA, RA, RA, RA, RA, HZ, R.

In [6]:
# Optimize

# Only use months before the break 
# try starting with Jan -March only
NUM_MONTHS = 3

# for each year, combine month's value a single list of values 
annual_vals = dict()
for year in multinomial_dict.items():
    years_values = []
    for month in year[1].items(): # iterate year values (month, values)
        years_values+=month[1] # the month values; 
        if(month[0] >= NUM_MONTHS): 
            annual_vals[year[0]] = years_values 
            break # only want the months less than April 

In [7]:
# Make a frequency dictionary of these values over each year

# step 1) for each year, for each type, set to zero 
# existing_weather_types includes only codes present in the data to reduce columns
freq_dict = dict()
existing_weather_types = ['PL', 'DU', 'DZ', 'RA', 'BLSN', 'SN', 'TS', 'FZFG', 'FZRA', 'FG', 'UP', 'BR', 'HZ']

for year in annual_vals.keys():
    freq_dict[year] = dict()
    for t in existing_weather_types: 
        freq_dict[year][t] = 0
        
# step 2) count values in dictionary created above 
for year, vals in annual_vals.items():
    for v in vals:
        freq_dict[year][v]+=1

freq_df = pd.DataFrame.from_dict(freq_dict).transpose();

# Attach class labels to freq_df by year
# since are from separate data sets
temp = []
for x in freq_dict.keys():    
    temp.append(int( labels[labels['Year'] == x].month))
    if x == 2020: break
temp.append('?') # 2021, 2022 are not present in the data set, fix later
temp.append('?')
freq_df['class_labels'] = temp

# dropped empty years/features in the data 
freq_df = freq_df.drop([2000,2001,2002,2003]).drop(['DU', 'TS'], axis=1)
freq_df = freq_df[:-2]

### Table 1a. Data as Frequency Table of weather types 

In [8]:
# now the data is in a usable format with the class label has been joined to each record
# records are annual entries  
# features are the count of each weather type, label attached in final column
freq_df

,PL,DZ,RA,BLSN,SN,FZFG,FZRA,FG,UP,BR,HZ,class_labels
1996,0,0,1,0,17,0,0,11,0,0,0,5
1997,0,0,0,0,7,0,0,0,0,6,0,4
1998,0,0,1,0,16,0,0,1,0,9,6,4
1999,0,0,0,0,31,0,0,0,0,31,24,4
2004,0,0,0,0,31,0,0,4,0,25,29,4
2005,0,0,4,0,27,6,1,8,69,26,13,4
2006,0,0,2,0,52,5,0,5,37,25,16,5
2007,2,2,47,0,49,4,0,5,1,30,23,4
2008,0,0,46,0,63,0,0,1,5,24,15,5
2009,0,0,27,1,53,3,0,5,7,40,26,5


In [9]:
# Avoid Pandas indexing headaches  

def Counts( f=None, c=None):
    table=class_freq.copy()
    if f == None:
        if c == None: return float(table[:].values.sum()) # count sample space
        else: return float(table.loc[:,c].values.sum())   # count c_j in space 
    if c == None: return float(table.loc[f,:].values.sum()) # count f_i in space 
    return float(table.loc[f,c])  # count f_i & c_j in space


In [10]:
# Reserve some years for the test set before training

# how many for test, train 
years = freq_df.index 
perc = 0.2 # Adjustable use percentage of years for the test set
num_test= int ( np.ceil (len(years)*perc) ) # remove this many records based on percentage 

# get random year indices for test, train
test = [] # get random indices in the data range
for i in random.sample(range(0, len(years)), num_test): test.append(years[i])
train =[y for y in years if y not in test]

# split records into test, train using indices above 
getTest = lambda x : trainingData[trainingData.index==test[x]] 
trainingData = freq_df
testData = pd.concat([getTest(i) for i in range(len(test))]) # get test set 
trainingData = trainingData.drop(test) # remove test set 

# data reserved for later testing 
# remaining data to use by classifer 
print( 'records in test set', len(testData), ', training set', len(trainingData) )

records in test set 5 , training set 16


In [11]:
# Create Likelihoods Table

# count feature, class combinations 
class_freq = pd.DataFrame(trainingData[trainingData['class_labels']==4].sum())
class_freq['5'] = trainingData[trainingData['class_labels']==5].sum()
class_freq = class_freq.drop('class_labels')
class_freq.columns = ['4', '5']

# compute probabilities for each feature, given each class
# P( for f in F | c ) for c in C

# Optimization 

# Laplace Smoothing
# there are some features which have no datapoint 
# use alpha smoothing to avoid having a zero likelihoods for new features being seen 
alpha = 0.01 # smoothing param
k = len(class_freq) # number of features ro use in denominator


likelihoods = class_freq.copy()

# start with the count of c,j found together in the data
# then divide them by each class summation
likelihoods+=alpha # add alpha to each feature
likelihoods['4'] /= (Counts(c='4')+(k*alpha)) # normalize for alpha in numerator
likelihoods['5'] /= (Counts(c='5')+(k*alpha))
print(likelihoods['4'].sum(), likelihoods['5'].sum() ) # alpha check:  should be approx 1 

1.0 0.9999999999999999


### Table 1b. Likelihoods Table P(F|C)

In [12]:
likelihoods

,4,5
PL,0.003555,0.000013
DZ,0.001782,0.000013
RA,0.210095,0.222628
BLSN,0.000009,0.001285
SN,0.251757,0.314218
FZFG,0.025716,0.027999
FZRA,0.006214,0.002557
FG,0.031034,0.044536
UP,0.116132,0.109412
BR,0.183502,0.176833


In [13]:
testData 

,PL,DZ,RA,BLSN,SN,FZFG,FZRA,FG,UP,BR,HZ,class_labels
2019,0,0,34,0,32,8,0,9,10,43,17,4
2009,0,0,27,1,53,3,0,5,7,40,26,5
1998,0,0,1,0,16,0,0,1,0,9,6,4
2004,0,0,0,0,31,0,0,4,0,25,29,4
2018,0,0,34,0,39,8,2,8,12,41,16,5


In [15]:
# Classifier 

# get probability variables
t = Counts(c='4') + Counts(c='5') # total 
y = [ (Counts(c='4') / t) , (Counts(c='5') / t ) ] # P(C)
l0 = likelihoods['4'] # P(x|c1)
l1 = likelihoods['5'] # P(x|c2)

# predicted vs true labels 
pred = ['?']*len(testData) # past two years in set are not yet labeled so start with '?'
true = [ str(x) for x in testData['class_labels'] ] 

# classifyt test set 
for i in range(len(testData)):
    
     # IGNORE the past 2 years are not yet known data 
    if testData.iloc[i,-1] == '?': continue
        
    t = testData.iloc[i,:-1] # testData point i
    
    # P(C|x)
    p4 = np.product([t*l for t,l in zip(t, l0) if t!=0])*y[0] #P(c1|x)
    p5 = np.product([t*l for t,l in zip(t, l1) if t!=0])*y[1] #P(c2|x)
    
    # if p4 is greater, pred class 4
    pred[i] = '4' if p4 > p5 else '5'

print('True labels', true)
print('Predicted: ', pred)
accuracy = sum( [ pred[x]==true[x] for x in range(len(testData)) ] ) / len(testData)
print(accuracy)

True labels ['4', '5', '4', '4', '5']
Predicted:  ['4', '5', '4', '4', '4']
0.8





## Classifier 2) Bayes with Gaussian Distribution

### Table 2a. The initial numerical features 

In [16]:
priors = { 
    4: labels['month'].value_counts()[4] / len(labels), 
    5: labels['month'].value_counts()[5] / len(labels)} 

In [17]:
# classifier function

def Gauss(x, mean, std):
    exponent = math.exp(-((x - mean) ** 2 / (2 * std ** 2)))
    return (1 /(math.sqrt(2 * math.pi) * std)) * exponent

In [18]:
# data 
monthly_numerical_features 

,DATE,max_temp,mean_temp,min_temp,station_pressure,x_month,Year,decimal_day_of_year,date_time,class_month,TOD,TOY
0,1998-05-31 00:00:00,60.5,48.6,36.6,29.27,5,1998,110.7042,04/20/1998 04:54:00 PM,4,0.7042,110.0
1,1998-10-31 00:00:00,34.3,26.0,17.6,29.27,10,1998,110.7042,04/20/1998 04:54:00 PM,4,0.7042,110.0
2,1998-09-30 00:00:00,53.9,45.1,36.2,29.28,9,1998,110.7042,04/20/1998 04:54:00 PM,4,0.7042,110.0
3,1998-07-31 00:00:00,70.5,60.5,50.4,29.44,7,1998,110.7042,04/20/1998 04:54:00 PM,4,0.7042,110.0
4,1998-11-30 00:00:00,16.9,9.0,1.1,29.21,11,1998,110.7042,04/20/1998 04:54:00 PM,4,0.7042,110.0
...,...,...,...,...,...,...,...,...,...,...,...,...
244,2020-02-29 00:00:00,8.7,-3.8,-16.3,29.42,2,2020,118.5389,04/27/2020 12:56:00 PM,4,0.5389,118.0
245,2020-01-31 00:00:00,-8.3,-16.1,-23.8,29.49,1,2020,118.5389,04/27/2020 12:56:00 PM,4,0.5389,118.0
246,2020-06-30 00:00:00,71.0,59.9,48.7,29.36,6,2020,118.5389,04/27/2020 12:56:00 PM,4,0.5389,118.0
247,2020-05-31 00:00:00,66.8,53.2,39.5,29.47,5,2020,118.5389,04/27/2020 12:56:00 PM,4,0.5389,118.0


In [19]:
# Optimize 

# Shouldn't use the year after the melt point, start with only Jan-Mar range for each year 
monthly_numerical_features = monthly_numerical_features[monthly_numerical_features['x_month'] <= 3]
monthly_numerical_features = monthly_numerical_features.reset_index(drop=True)

In [20]:
# reserve some years for the test set 
# same process as above 

years = monthly_numerical_features['Year'].unique()
perc = 0.2 # use percentage of years for the test set
num_test= int ( np.ceil (len(years)*perc) ) 

# test and train will be an index mappings to the data 
# remove some years at random 
test = [] # get a random year in the data range
for i in random.sample(range(0, len(years)), num_test): test.append(years[i])

train =[y for y in years if y not in test]

trainingData = monthly_numerical_features

testData = pd.DataFrame(columns=trainingData.columns)
for i in test: 
    testData = pd.concat([ testData, monthly_numerical_features[monthly_numerical_features['Year']==i]])
trainingData = trainingData.drop(list(testData.index))

trainingData.reset_index(drop=True)
testData.reset_index(drop=True)
print(len(trainingData), len(testData))

50 14


In [21]:
# get parameters

feats = ['station_pressure', 'mean_temp'] 
cls = [4,5]

# features {mu, sig}
f_params = dict()
for f in feats :
    f_params[f] = (trainingData[f].mean() , trainingData[f].std())
    
# reduce by class { mu, sig}
cf_params = dict()
for c in cls:
    cf_params[c] = dict()
    for f in feats:
        cf_params[c][f] = (trainingData[ trainingData['class_month']==c ][f].mean() , 
             trainingData[ trainingData['class_month']==c ][f].std() )
        
# F { mu, sig}
print('F params: ', f_params)

# F|C { mu, sig}
print('F|c params:', cf_params)


F params:  {'station_pressure': (29.41300000000001, 0.18548612358021924), 'mean_temp': (4.638000000000001, 10.77815042252669)}
F|c params: {4: {'station_pressure': (29.389354838709686, 0.1773684579976756), 'mean_temp': (4.680645161290322, 11.411906628746392)}, 5: {'station_pressure': (29.45157894736842, 0.19670234585661914), 'mean_temp': (4.56842105263158, 9.958695398248091)}}


In [22]:
# Classify 

# get Data points , get value (f1, f2) 
X = [] 
for i in zip( testData[feats[0]] , testData[feats[1]]): 
    X.append(i)

# Predict
# given the  f ( mu , sig ) for this x, 
# what is the label? 
    
pred = []

for x in X:
    t = [] # accumulate class prediction 
    for c in (4,5):
        p=1 
        
        # multiply datapoint by parameters 
        for f_ind in range(len(feats)): 
            f = Gauss(x[f_ind] , f_params[feats[f_ind]][0], f_params[feats[f_ind]][1]) 
            fc = Gauss(x[f_ind] , cf_params[c][feats[f_ind]][0], cf_params[c][feats[f_ind]][1]) 
            p*=f*fc
            
        # multiply by prior
        p*=priors[c]
        t.append(p)
        
    # the greater one is the prediction 
    if(t[0] > t[1]): 
        pred.append(4) 
    else: pred.append(5)

In [23]:
# Accuracy

true = testData['class_month'].values
print('true', list(true))
print('pred', pred)

# Accuracy
sum( [ pred[i] == true[i] for i in range(len(pred)) ] ) / len(true)

true [4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4]
pred [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


0.6428571428571429

## Classifier 3 Bayes With Hidden Markov 

### Table 3a.Data Table showing sequential weather patterns  

In [24]:
# go through imported Multinomial dictionary to 
# form the most recent observation chain for each year
temp_col, temp_val=[], [] 
for year, x in annual_vals.items():
    # the chains must contain 20 observations
    if(len(x))>=20:
        temp_col.append(year)
        temp_val.append(x[-20:])
        
chains = pd.DataFrame( {temp_col[x] : temp_val[x] for x in range(len(temp_col))} )
chains

,1996,1998,1999,2004,2005,2006,2007,2008,2009,2010,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,SN,HZ,SN,SN,UP,UP,RA,RA,SN,SN,...,SN,SN,UP,UP,BR,SN,BR,BR,UP,SN
1,SN,SN,BR,FG,SN,SN,RA,BR,RA,BR,...,UP,BR,RA,SN,HZ,BR,HZ,SN,UP,FZFG
2,FG,BR,SN,BR,UP,UP,SN,SN,RA,RA,...,SN,HZ,RA,BR,HZ,HZ,SN,BR,SN,BR
3,FG,HZ,BR,HZ,SN,SN,RA,RA,SN,SN,...,SN,SN,RA,UP,UP,RA,FZFG,RA,FZFG,FG
4,FG,RA,SN,SN,UP,UP,SN,SN,SN,RA,...,HZ,BR,RA,HZ,SN,RA,BR,RA,BR,SN
5,FG,SN,BR,FG,UP,UP,BR,SN,UP,RA,...,UP,SN,RA,RA,HZ,SN,RA,RA,UP,FZFG
6,SN,BR,SN,BR,UP,SN,HZ,SN,RA,RA,...,UP,SN,UP,SN,RA,SN,FG,RA,HZ,BR
7,FG,SN,BR,HZ,UP,UP,RA,BR,SN,UP,...,RA,UP,RA,UP,RA,SN,SN,BR,FG,HZ
8,SN,BR,SN,SN,UP,HZ,SN,RA,UP,RA,...,SN,HZ,SN,HZ,RA,SN,BR,RA,SN,FG
9,SN,SN,BR,HZ,SN,SN,BR,SN,HZ,RA,...,SN,SN,SN,HZ,RA,RA,RA,SN,BR,SN


In [25]:
# get all unique observation types existing in chains 
obs= set()
for x in chains:
    for y in chains[x].values:
        obs.add(y)
obs

{'BR', 'FG', 'FZFG', 'FZRA', 'HZ', 'PL', 'RA', 'SN', 'UP'}

### Table 3b. Transitional Probabilities to find param: A_{i,<=j}

In [26]:
trans_dict = dict.fromkeys(obs)
for o in obs: trans_dict[o] = dict.fromkeys(obs, 0)
    
# columns are years, records 
for c in chains.columns:
    # compare each weather state with the proceeding
    for i in range(len(chains[c])-1): 
        # increment transition count in dictionary 
        trans_dict[chains[c][i]][chains[c][i+1]]+=1
        
trans_freq_df = pd.DataFrame.from_dict(trans_dict)
# column -> row : freq 
trans_freq_df

,FZRA,HZ,FG,UP,BR,SN,RA,PL,FZFG
FZRA,0,0,0,1,1,0,1,0,0
HZ,0,4,0,9,16,6,1,0,0
FG,0,2,3,1,2,6,2,0,0
UP,1,2,1,11,14,13,7,0,0
BR,0,0,3,0,0,48,7,0,7
SN,1,17,7,16,16,36,29,0,0
RA,0,9,1,13,14,13,48,1,0
PL,0,1,0,0,0,0,1,0,0
FZFG,0,1,0,0,0,6,0,0,0


In [27]:
obs_totals = [sum(trans_freq_df[c]) for c in trans_freq_df.columns]
obs_totals

[2, 36, 15, 51, 63, 128, 96, 1, 7]

### Table 3c (as probabilities)

In [28]:
# transition from column -> row probabilities
# trans_prob_df['HZ']['RA'] = 0.25 ( 'HZ'-> 'RA' )
trans_prob_df = pd.DataFrame(trans_freq_df)
for i in range(len(trans_prob_df)):
    trans_prob_df.iloc[:,i] /= obs_totals[i] 
trans_prob_df

,FZRA,HZ,FG,UP,BR,SN,RA,PL,FZFG
FZRA,0.0,0.000000,0.000000,0.019608,0.015873,0.000000,0.010417,0.0,0.0
HZ,0.0,0.111111,0.000000,0.176471,0.253968,0.046875,0.010417,0.0,0.0
FG,0.0,0.055556,0.200000,0.019608,0.031746,0.046875,0.020833,0.0,0.0
UP,0.5,0.055556,0.066667,0.215686,0.222222,0.101562,0.072917,0.0,0.0
BR,0.0,0.000000,0.200000,0.000000,0.000000,0.375000,0.072917,0.0,1.0
SN,0.5,0.472222,0.466667,0.313725,0.253968,0.281250,0.302083,0.0,0.0
RA,0.0,0.250000,0.066667,0.254902,0.222222,0.101562,0.500000,1.0,0.0
PL,0.0,0.027778,0.000000,0.000000,0.000000,0.000000,0.010417,0.0,0.0
FZFG,0.0,0.027778,0.000000,0.000000,0.000000,0.046875,0.000000,0.0,0.0


### Table 3d) Initial Probabilities Table for HMM (PI param)

In [29]:
# Pi, the intiial probabilities of each state
PI = pd.DataFrame({ trans_prob_df.columns[x] : 
                   [obs_totals[x]/ (sum(obs_totals))] for x in range(len(trans_prob_df.columns)) })
PI

,FZRA,HZ,FG,UP,BR,SN,RA,PL,FZFG
0,0.005013,0.090226,0.037594,0.12782,0.157895,0.320802,0.240602,0.002506,0.017544
